In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.functional as F
import pickle
import re
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import random
from tqdm.auto import tqdm

RANDOM_STATE = 5411
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device="cpu"

In [2]:
torch.cuda.manual_seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
# torch.cuda.set_device(device)

In [3]:
data = pickle.load(open("data/data.pkl", "rb"))

In [4]:
def preprocess(text):
    text = re.sub(r"@[A-Za-z0-9]+", "", text)
    text = re.sub(r"[0-9]+", "", text)
    text = re.sub(r"#[A-Za-z0-9]*", "", text)
    text = re.sub(r"https?:\/\/\S+", "", text)
    text = re.sub(r"[\n]", " ", text)
    # text = re.sub(r"[.*/]", " ", text)
    text = text.encode("ascii", "ignore").decode("ascii")
    return text.strip()

In [5]:
data = [[preprocess(i[0]), i[1]] for i in data]

In [6]:
tokenizer = get_tokenizer("spacy")

c:\Users\aritr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [7]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [8]:
vocab = build_vocab_from_iterator(yield_tokens(data), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [9]:
len(vocab)

23160

In [10]:
stoi = lambda x: vocab(tokenizer(x))

In [11]:
vocab.get_itos()[0]

'<unk>'

In [12]:
tokenizer("<unk>")

['<', 'unk', '>']

In [6]:
import pandas as pd

glove = pd.read_csv('D:/Downloads/glove.840B.300d/glove.840B.300d.txt', sep=" ", quoting=3, header=None, index_col=0)

In [12]:
glove.head(5)

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
",",-0.082752,0.672040,-0.14987,-0.064983,0.056491,0.402280,0.002775,-0.331100,-0.306910,2.0817,...,-0.14331,0.018267,-0.18643,0.207090,-0.355980,0.053380,-0.050821,-0.191800,-0.378460,-0.06589
.,0.012001,0.207510,-0.12578,-0.593250,0.125250,0.159750,0.137480,-0.331570,-0.136940,1.7893,...,0.16165,-0.066737,-0.29556,0.022612,-0.281350,0.063500,0.140190,0.138710,-0.360490,-0.03500
the,0.272040,-0.062030,-0.18840,0.023225,-0.018158,0.006719,-0.138770,0.177080,0.177090,2.5882,...,-0.42810,0.168990,0.22511,-0.285570,-0.102800,-0.018168,0.114070,0.130150,-0.183170,0.13230
and,-0.185670,0.066008,-0.25209,-0.117250,0.265130,0.064908,0.122910,-0.093979,0.024321,2.4926,...,-0.59396,-0.097729,0.20072,0.170550,-0.004736,-0.039709,0.324980,-0.023452,0.123020,0.33120
to,0.319240,0.063160,-0.27858,0.261200,0.079248,-0.214620,-0.104950,0.154950,-0.033530,2.4834,...,-0.12977,0.371300,0.18888,-0.004274,-0.106450,-0.258100,-0.044629,0.082745,0.097801,0.25045


In [13]:

glove_embedding = {key: val.values for key, val in glove.T.items()}

In [15]:
len(glove_embedding[','])

300

In [31]:
stoi(",")

[3]

In [29]:
embedding_matrix = torch.zeros((len(vocab), 300))

for i in range(1, len(vocab)):
    word = vocab.get_itos()[i]
    if word in glove_embedding:
        embedding_matrix[i]=torch.from_numpy(glove_embedding[word])

In [30]:
embedding_matrix[:5]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1941,  0.2260, -0.4376,  ...,  0.0920,  0.3863,  0.1174],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        [-0.0828,  0.6720, -0.1499,  ..., -0.1918, -0.3785, -0.0659],
        [ 0.3192,  0.0632, -0.2786,  ...,  0.0827,  0.0978,  0.2504]])

In [35]:
pickle.dump(embedding_matrix,open("embeddings.pkl","wb"))

In [36]:
x=pickle.load(open("embeddings.pkl","rb"))

In [37]:
type(x)

torch.Tensor

In [13]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]

    for _text, _label in batch:
        label_list.append(_label)
        processed_text = torch.tensor(stoi(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

In [14]:
data = random.sample(data, len(data))

In [15]:
split = 0.2
split_index = int(len(data) * (1 - split))

In [16]:
train_data, test_data = data[:split_index], data[split_index:]

In [17]:
type(train_data)

list

In [18]:
len(train_data), len(test_data)

(2257, 565)

In [19]:
BATCH_SIZE = 16

In [20]:
train_dataloader = DataLoader(
    train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch,generator=torch.Generator(device=device)
)

In [21]:
test_dataloader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch,
    generator=torch.Generator(device=device),
)

In [22]:
from torch import nn


class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_nodes, num_class):
        super(TextClassificationModel, self).__init__()

        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.dense_block = nn.Sequential(
            nn.Linear(embed_dim, num_nodes),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(num_nodes, num_class),
        )

        self.embedding.apply(self.init_weights)
        self.dense_block.apply(self.init_weights)

    def init_weights(self, layer):
        initrange = 0.5

        if isinstance(layer, nn.Linear):
            layer.weight.data.uniform_(-initrange, initrange)
            layer.bias.data.zero_()
        elif isinstance(layer, nn.EmbeddingBag):
            layer.weight.data.uniform_(-initrange, initrange)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.dense_block(embedded)

In [23]:
num_class = 2
vocab_size = len(vocab)
embed_size = 64
model = TextClassificationModel(vocab_size, embed_size, 512, num_class).to(device)

In [34]:
optimizer.param_groups

[{'params': [Parameter containing:
   tensor([[ 5.0617e-40,  1.3606e-15, -2.2866e-40,  ..., -4.2641e-17,
             3.1035e-39, -6.7272e-22],
           [-8.5938e-02,  3.2254e-01,  5.0457e-02,  ..., -3.8322e-02,
            -8.9960e-02, -4.4433e-03],
           [ 1.2289e-01,  6.0114e-02,  1.8644e-01,  ..., -2.9667e-01,
            -2.7566e-01, -1.9465e-01],
           ...,
           [-9.0253e-06,  6.6264e-05, -3.1442e-04,  ...,  1.6112e-04,
             3.4517e-05,  6.5777e-07],
           [ 6.4758e-40, -2.0265e-12,  6.6061e-41,  ..., -1.3085e-18,
            -3.8977e-39, -4.2688e-40],
           [ 4.9056e-05, -3.1769e-04, -2.2225e-04,  ...,  1.2772e-04,
             1.0306e-04, -6.7761e-05]], requires_grad=True),
   Parameter containing:
   tensor([[ 4.9720e-03, -3.8873e-03,  2.8663e-03,  ..., -3.0103e-04,
            -5.1750e-04,  5.6413e-03],
           [ 1.6180e-03, -5.2169e-02, -1.9099e-02,  ...,  1.8352e-02,
            -3.8149e-02, -1.2734e-02],
           [ 4.3576e-03, -6.95

In [35]:
for params in optimizer.param_groups:
    params["lr"] *= 0.1

In [36]:
optimizer.param_groups

[{'params': [Parameter containing:
   tensor([[ 5.0617e-40,  1.3606e-15, -2.2866e-40,  ..., -4.2641e-17,
             3.1035e-39, -6.7272e-22],
           [-8.5938e-02,  3.2254e-01,  5.0457e-02,  ..., -3.8322e-02,
            -8.9960e-02, -4.4433e-03],
           [ 1.2289e-01,  6.0114e-02,  1.8644e-01,  ..., -2.9667e-01,
            -2.7566e-01, -1.9465e-01],
           ...,
           [-9.0253e-06,  6.6264e-05, -3.1442e-04,  ...,  1.6112e-04,
             3.4517e-05,  6.5777e-07],
           [ 6.4758e-40, -2.0265e-12,  6.6061e-41,  ..., -1.3085e-18,
            -3.8977e-39, -4.2688e-40],
           [ 4.9056e-05, -3.1769e-04, -2.2225e-04,  ...,  1.2772e-04,
             1.0306e-04, -6.7761e-05]], requires_grad=True),
   Parameter containing:
   tensor([[ 4.9720e-03, -3.8873e-03,  2.8663e-03,  ..., -3.0103e-04,
            -5.1750e-04,  5.6413e-03],
           [ 1.6180e-03, -5.2169e-02, -1.9099e-02,  ...,  1.8352e-02,
            -3.8149e-02, -1.2734e-02],
           [ 4.3576e-03, -6.95

In [25]:
EPOCHS = 20
LR = 0.001

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [27]:
def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
):
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (y, X, offsets) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X, offsets)

        # 2. Calculating and accumulating loss
        loss = loss_fn(y_pred, y)

        l2_lambda = 0.001
        l2_regularization = torch.tensor(0.0, requires_grad=True)
        for name, param in model.named_parameters():
            if "bias" not in name:
                l2_regularization = l2_regularization + torch.norm(param, p=2)

        loss = loss + l2_lambda * l2_regularization

        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(y_pred, dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)
        # print(y_pred, y_pred_class, train_prec, "\n")
        # break

    # Adjust metrics to get average loss and accuracy per batch
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)

    return train_loss, train_acc


def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device,
):
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (y, X, offsets) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X, offsets)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)

            l2_lambda = 0.001
            l2_regularization = torch.tensor(0.0, requires_grad=True)
            for name, param in model.named_parameters():
                if "bias" not in name:
                    l2_regularization = l2_regularization + torch.norm(param, p=2)
            loss = loss + l2_lambda * l2_regularization

            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

            # break

    # Adjust metrics to get average loss and accuracy per batch
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

    return test_loss, test_acc


def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device,
):
    # Create empty results dictionary
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
    }

    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        test_loss, test_acc = test_step(
            model=model, dataloader=test_dataloader, loss_fn=loss_fn, device=device
        )

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

In [28]:
next(iter(train_dataloader))

(tensor([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0]),
 tensor([ 130,   51,   73,  ...,   13, 1053,   36]),
 tensor([   0,  328,  567,  819, 1168, 1623, 1848, 2067, 2396, 2469, 2712, 3086,
         5497, 5501, 5829, 5872]))

In [29]:
train(model, train_dataloader, test_dataloader, optimizer, criterion, EPOCHS, device)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9677 | train_acc: 0.6699 | test_loss: 0.8158 | test_acc: 0.7250
Epoch: 2 | train_loss: 0.7218 | train_acc: 0.7746 | test_loss: 0.6284 | test_acc: 0.8000
Epoch: 3 | train_loss: 0.5261 | train_acc: 0.8521 | test_loss: 0.5107 | test_acc: 0.8385
Epoch: 4 | train_loss: 0.4067 | train_acc: 0.8856 | test_loss: 0.4578 | test_acc: 0.8472
Epoch: 5 | train_loss: 0.3175 | train_acc: 0.9217 | test_loss: 0.4226 | test_acc: 0.8347
Epoch: 6 | train_loss: 0.2737 | train_acc: 0.9252 | test_loss: 0.4344 | test_acc: 0.8503
Epoch: 7 | train_loss: 0.2363 | train_acc: 0.9423 | test_loss: 0.4124 | test_acc: 0.8438
Epoch: 8 | train_loss: 0.2158 | train_acc: 0.9476 | test_loss: 0.4307 | test_acc: 0.8500
Epoch: 9 | train_loss: 0.1961 | train_acc: 0.9551 | test_loss: 0.4146 | test_acc: 0.8507
Epoch: 10 | train_loss: 0.1829 | train_acc: 0.9608 | test_loss: 0.4236 | test_acc: 0.8434
Epoch: 11 | train_loss: 0.1760 | train_acc: 0.9595 | test_loss: 0.4263 | test_acc: 0.8420
Epoch: 12 | train_l

{'train_loss': [0.9676783890791343,
  0.7217654415839155,
  0.5261246675336865,
  0.4067236065444812,
  0.31747695845617374,
  0.27368229207858236,
  0.236286936563925,
  0.21580671445584634,
  0.19610047057061128,
  0.18292804183044903,
  0.17596764253898406,
  0.16238376936337479,
  0.15628138766951963,
  0.15347192315778263,
  0.1489973974899507,
  0.14218394961995137,
  0.14088348419943325,
  0.13323462298008756,
  0.13320934601967604,
  0.1312231827842098],
 'train_acc': [0.6698943661971831,
  0.7746478873239436,
  0.852112676056338,
  0.8855633802816901,
  0.9216549295774648,
  0.9251760563380281,
  0.9423415492957746,
  0.9476232394366197,
  0.9551056338028169,
  0.9608274647887324,
  0.9595070422535211,
  0.9647887323943662,
  0.9691901408450704,
  0.96875,
  0.9722711267605634,
  0.9744718309859155,
  0.971830985915493,
  0.9744718309859155,
  0.9735915492957746,
  0.9775528169014085],
 'test_loss': [0.8157598922650019,
  0.6283977934055858,
  0.5106584520803558,
  0.457751014

In [27]:
from torchmetrics.classification import F1Score

f1_fn = F1Score(task="binary", num_labels=5)

In [28]:
type(f1_fn)

torchmetrics.classification.f_beta.BinaryF1Score

In [ ]:
x=torch.rand((16,2))

In [ ]:
x.dtype

In [ ]:
y,X,offset=collate_batch([["The sun rises in the east and sets in the west.",0]])

In [ ]:
y,X,offset

In [ ]:
new_model.eval()

# Setup test loss and test accuracy values
test_loss, test_acc = 0, 0

# Turn on inference context manager
with torch.inference_mode():
    
    # Send data to target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    test_pred_logits = new_model(X, offset)

    # 2. Calculate and accumulate loss
    # loss = loss_fn(test_pred_logits, y)
    # test_loss += loss.item()

    # Calculate and accumulate accuracy
    test_pred_labels = test_pred_logits.argmax(dim=1)
    # test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

In [ ]:
test_pred_labels,test_pred_logits

In [ ]:
# from pathlib import Path

# def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
#     # Create target directory
#     target_dir_path = Path(target_dir)
#     target_dir_path.mkdir(parents=True, exist_ok=True)

#     # Create model save path
#     model_save_path = target_dir_path / model_name

#     # Save the model state_dict()
#     print(f" Saving model to: {model_save_path}")
#     torch.save(obj=model.state_dict(), f=model_save_path)
    
# save_model(
#     model=model, target_dir="models", model_name=f"first_model.pth"
# )


In [ ]:
new_model = TextClassificationModel(vocab_size, embed_size, num_class).to(device)
new_model.load_state_dict(torch.load("models/first_model.pth"))

In [ ]:
import pickle

data = pickle.load(open("data.pkl", "rb"))
rules = open("samples.txt", "r").readlines()
rules = [i[0:-1] for i in rules]
exemplars=pickle.load(open("samples.pkl", "rb"))

In [ ]:
for text,label in data[:16]:
    print(label)

In [ ]:
from data_setup import get_dataloaders

tr_dl, ts_dl, vl = get_dataloaders("data.pkl", 16)

In [ ]:
x=next(iter(tr_dl))

In [ ]:
x[0].unsqueeze(1).shape

In [ ]:
# for i,j in zip(rules,exemplars):
#     print(i," ",j)

In [ ]:
import re

y = []

for text, label in data[:]:
    temp = []
    for i in range(len(rules)):
        compiled_pattern = re.compile(rules[i], re.IGNORECASE)
        if bool(compiled_pattern.search(text)) and label == 0:
            temp.append(0)
        else:
            temp.append(1)
    y.append(temp)

In [ ]:
len(y[0]),len(y)

In [ ]:
data=[i for i,j in data]

In [ ]:
rule_data = [[data[i],y[i]] for i in range(2822)]

In [ ]:
len(rule_data),len(rule_data[0]),len(rule_data[4][1])

In [ ]:
pickle.dump(rule_data,open("rule_data.pkl","wb"))

In [ ]:
a=pickle.load(open("rule_data.pkl", "rb"))

In [ ]:
len(a),len(a[0])

In [ ]:
for i in a:
    print(i[0],i[1])
    break

In [ ]:
data = [[preprocess(i[0]), i[1]] for i in a]

In [ ]:
a[0][0]

In [ ]:
def preprocess(text):
    text = re.sub(r"@[A-Za-z0-9]+", "", text)
    text = re.sub(r"[0-9]+", "", text)
    text = re.sub(r"#[A-Za-z0-9]*", "", text)
    text = re.sub(r"https?:\/\/\S+", "", text)
    text = re.sub(r"[\n]", " ", text)
    text = re.sub(r"['\",*%$#()]", " ", text)
    text = re.sub(r"[.]", " . ", text)
    text = text.encode("ascii", "ignore").decode("ascii")
    return text.strip()

In [ ]:
data = [[preprocess(i[0]), i[1]] for i in rule_data]

In [ ]:
len(data),len(data[0]),len(data[0][1])

In [ ]:
from data_setup import get_dataloaders

train_dl,test_dl,vl=get_dataloaders("rule_data.pkl",16)

In [ ]:
y, x, o = next(iter(train_dl))

In [ ]:
y.shape,x.shape

In [ ]:
4460/16

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch.set_default_device(device)

In [ ]:
for b,d in enumerate(train_dl):
    print(b,d,d[1].shape)
    break

In [ ]:
d[0].shape,d[1].shape,d[2].shape

In [ ]:
import torch
y=torch.rand((16,43))
y_pred=torch.rand((16,43))

In [ ]:
temp=y-y_pred

In [ ]:
c=0
for i in temp.reshape(-1):
    c+=1

In [ ]:
c = 0
num_classes = 5
for i in range(num_classes):
    if (y[i] - y_pred[i]).abs() < 0.5:
        c += 1
acc = c / num_classes
acc

In [15]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim.adam
import torch
import torch.nn as nn

In [16]:
class ClassificationNetwork(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_nodes, num_class, embedding_matrix):
        super(ClassificationNetwork, self).__init__()

        # self.glove_embedding = nn.EmbeddingBag.from_pretrained(
        #     embedding_matrix, freeze=True
        # )
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.dense_block = nn.Sequential(
            nn.Linear(embed_dim, num_nodes),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(num_nodes, num_class),
        )

        self.embedding.apply(self.init_weights)
        self.dense_block.apply(self.init_weights)

    def init_weights(self, layer):
        initrange = 0.5

        if isinstance(layer, nn.Linear or nn.EmbeddingBag):
            layer.weight.data.uniform_(-initrange, initrange)
            layer.bias.data.zero_()

    def forward(self, text, offsets):
        # combined_embedding = torch.cat(
        #     [self.embedding(text, offsets), self.glove_embedding(text, offsets)],
        #     dim=-1,
        # )
        embedded = self.embedding(text, offsets)
        return self.dense_block(embedded)


In [19]:
model = ClassificationNetwork(500, 20, 50, 2, None)


optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = ReduceLROnPlateau(optimizer, "max", patience=3, verbose=True)


In [20]:
type(scheduler)

torch.optim.lr_scheduler.ReduceLROnPlateau

In [38]:
import pickle

res1=pickle.load(open("results\classification_model_dropout0.6_2508_results.pkl","rb"))
res2=pickle.load(open("results\classification_model_dropout0.5_2508_results.pkl","rb"))

In [39]:
import pandas as pd

In [44]:
pd.DataFrame(res1).sort_values(by="test_f1",ascending=False),pd.DataFrame(res2).sort_values(by="test_f1",ascending=False)

(    train_loss  train_f1  train_acc  test_loss   test_f1  test_acc
 17    0.037894  0.989567   0.990423   0.343799  0.887422  0.897327
 11    0.112145  0.957060   0.962558   0.311856  0.884228  0.892610
 13    0.082809  0.968599   0.970766   0.325911  0.881725  0.896148
 18    0.033232  0.988797   0.990423   0.355096  0.880266  0.887893
 12    0.086164  0.967595   0.969758   0.316964  0.876140  0.893789
 10    0.135249  0.948615   0.953125   0.319740  0.875109  0.883019
 8     0.173269  0.921297   0.927275   0.317038  0.871735  0.883019
 14    0.073092  0.970906   0.973286   0.338545  0.867581  0.890252
 19    0.036350  0.991695   0.992440   0.374309  0.865950  0.878302
 16    0.049480  0.980592   0.984375   0.349396  0.865330  0.887972
 9     0.158399  0.932475   0.941028   0.306827  0.863489  0.877123
 15    0.055742  0.982230   0.981351   0.349301  0.862266  0.893789
 7     0.225380  0.899866   0.909634   0.325797  0.853238  0.879481
 6     0.248921  0.887587   0.900562   0.334898 